<a href="https://colab.research.google.com/github/GeorgeShibanin/Galaxy_masses_estimation/blob/main/HDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import seaborn as sns
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error, mean_squared_error, explained_variance_score, mean_absolute_error, mean_squared_log_error, median_absolute_error, r2_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.ensemble import RandomForestRegressor
from tqdm import tqdm_notebook
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import normalized_mutual_info_score

from astropy.coordinates import Distance
from astropy import units as u
from astropy.cosmology import Planck18_arXiv_v2

In [3]:
def set_settings():
    tf.enable_v2_behavior()
    sns.reset_defaults()
    sns.set_context(context='talk',font_scale=0.7)
    tfd = tfp.distributions

In [4]:
def mount():
    drive.mount('/content/drive')

In [5]:
def check_gpu_device():
    if tf.test.gpu_device_name() != '/device:GPU:0':
        print('WARNING: GPU device not found.')
    else:
        print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

In [6]:
def prepare():
    set_settings()
    mount()
    check_gpu_device()

In [7]:
def load():
    data = pd.read_csv('drive/MyDrive/rcsed_z_less_03.csv')
    masses = pd.read_csv('drive/MyDrive/rcsed_z_less_03_logMstar_gal.csv')
    return data, masses

In [8]:
prepare()

Mounted at /content/drive
SUCCESS: Found GPU: /device:GPU:0


In [9]:
data, masses = load()

In [10]:
g = data['g']

In [11]:
kcorr_g = data['kcorr_g']

In [12]:
r = data['r']

In [13]:
kcorr_r = data['kcorr_r']

In [16]:
data

,Unnamed: 0,r2id_obj,ra_obj,dec_obj,redshift,amaj,ell,pa,fuv,fuv_err,fuv_size,kcorr_fuv,nuv,nuv_err,nuv_size,kcorr_nuv,u,u_err,u_size,kcorr_u,g,g_err,g_size,kcorr_g,r,r_err,r_size,kcorr_r,i,i_err,i_size,kcorr_i,z,z_err,z_size,kcorr_z,y,y_err,kcorr_y,j,j_err,kcorr_j,h,h_err,kcorr_h,k,k_err,kcorr_k,w1,w1_err,w2,w2_err,w3,w3_err,w4,w4_err
0,11,4964732,119.429449,52.995309,0.169052,5.174065,0.147545,75.428020,NaN,NaN,NaN,NaN,23.411406,0.803679,5.977952,0.545470,19.658782,0.186940,4.173847,0.294785,18.553901,0.030362,2.375188,0.515064,17.386399,0.020019,2.205640,0.168766,16.935629,0.025175,2.001972,0.083567,16.613500,0.051524,1.973587,0.039130,NaN,NaN,NaN,16.235859,0.030124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.262477,0.047,17.720986,0.048,17.344103,NaN,15.125856,NaN
1,19,4984675,171.950334,34.842062,0.042331,7.208284,0.742261,167.488800,20.772176,0.291849,3.5000,-0.023631,20.410298,0.118091,3.500000,0.009749,18.764511,0.122624,4.832413,0.121685,17.986342,0.020978,3.349131,0.053598,17.433048,0.020732,3.363956,0.036475,17.175305,0.025925,3.130031,0.000419,16.878486,0.074337,3.337605,0.011275,NaN,NaN,NaN,16.811426,0.069116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.180280,0.031,18.739711,0.047,16.582489,0.178,15.513918,NaN
2,21,4929008,229.317800,10.181280,0.059361,2.528525,0.547619,88.666280,NaN,NaN,NaN,NaN,21.859837,0.369446,3.500000,0.190795,18.780715,0.046751,1.218059,0.180941,17.115493,0.013880,1.169020,0.101875,16.451456,0.013966,1.163245,0.047751,16.146245,0.014310,1.153531,0.011143,15.904148,0.016994,1.180392,0.007042,15.451216,0.007531,0.018961,15.260725,0.007872,-0.025315,15.072135,0.011905,0.003010,15.228353,0.011539,-0.126780,16.290263,0.014,16.903854,0.012,17.320075,NaN,15.222852,NaN
3,36,5085135,2.593309,18.374851,0.171086,7.360223,0.357204,18.848080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.980644,1.283242,4.630747,-0.024583,22.601471,1.094536,1.439304,-0.029259,20.621648,0.139686,1.947570,-0.120074,19.969107,0.168316,1.497659,-0.223952,19.933580,0.531365,1.227112,0.019985,NaN,NaN,NaN,19.399256,0.210281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.738589,0.037,19.273824,0.071,16.601845,NaN,14.857815,NaN
4,40,4953301,223.510943,30.590311,0.105039,3.675426,0.594037,79.198190,21.593038,0.334868,3.5000,-0.004249,21.018972,0.209499,3.500000,0.009463,19.679360,0.097814,1.821242,0.161748,18.287194,0.012582,1.791371,0.139702,17.605897,0.010579,1.730272,0.046276,17.199028,0.011134,1.591608,-0.000667,16.847751,0.032656,1.641510,0.021436,NaN,NaN,NaN,16.523225,0.040377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.947873,0.071,16.945460,0.065,14.929433,0.029,13.848909,0.090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1652055,4712456,882789,41.771824,-26.326319,0.039340,4.282000,0.185675,49.725961,22.127898,0.225285,3.8083,0.182576,21.022361,0.098852,3.500000,NaN,NaN,NaN,NaN,NaN,20.012959,0.104666,NaN,0.608065,18.899818,0.038716,NaN,0.143761,18.201648,0.006626,3.960000,0.021834,18.305663,0.022567,NaN,0.015086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.953354,0.021,18.249756,0.027,NaN,NaN,NaN,NaN
1652056,4712457,3809143,2.118993,17.437530,0.067905,7.530045,0.267341,-0.020800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.384469,0.083983,5.173516,0.201118,16.820296,0.011034,3.166468,0.163659,15.940846,0.008213,3.125782,0.063800,15.565904,0.007487,2.890614,0.041873,15.273295,0.019930,2.757155,0.016281,NaN,NaN,NaN,15.035498,0.012558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.058192,0.469,14.695871,NaN
1652057,4712458,862747,10.077710,-29.285906,0.264760,4.611480,0.013327,55.280522,21.737172,0.455351,3.5000,0.118857,20.832204,0.188371,3.500000,NaN,NaN,NaN,NaN,NaN,18.657043,0.031068,NaN,0.870919,17.341057,0.009339,NaN,0.261753,16.846022,0.003620,4.620000,0.090772,16.548414,0.004510,NaN,0.097325,16.418114,0.010377,0.107160,16.157946,0.007116,-0.057447,15.909260,0.009649,-0.044925,15.693851,0.009513,-0.488879,17.

In [15]:
def prepare_data(data, masses):
    mask = ~masses['logMstar_gal'].isna()
    y = masses[mask]
    y = y.reset_index()
    y = y.drop(columns=['index'])
    data = data.drop(columns=['Unnamed: 0'])
    X = data[mask]
    X = X.reset_index()
    X = X.drop(columns=['index'])
    X = X.fillna(X.mean())
    return data, X, y, mask

In [19]:
def scale_data(X_train, X_test):
    scal = preprocessing.RobustScaler()
    X_train = scal.fit_transform(X_train)
    X_test = scal.transform(X_test)
    return X_train, X_test

In [17]:
data, X, y, mask = prepare_data(data, masses)

In [42]:
imp_data = data[['kcorr_g', 'w2']]

In [43]:
imp_data['ra_obj'] = data['ra_obj']
imp_data['dec_obj'] = data['dec_obj']
imp_data['z'] = data['z']
imp_data['redshift'] = data['redshift']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [23]:
!pip install hdbscan

     |████████████████████████████████| 6.4MB 7.3MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311695 sha256=602d614ad26f2dd2bc14de7c8ff1ed007f15fcc871de48f82601f487a6f7dcd1
  Stored in directory: /root/.cache/pip/wheels/42/63/fb/314ad6c3b270887a3ecb588b8e5aac50b0fad38ff89bb6dff2
Successfully built hdbscan


In [24]:
import hdbscan

In [25]:
def distance_to_center(x,y,z,label,centers):
  if(label==-1):
    return 0

  cluster=centers[int(label)]

  galaxy_coor=np.array([x,y,z])
  return np.linalg.norm(galaxy_coor-cluster)

def closest_center(x,y,z,search_tree):
  point=np.array([x,y,z])
  return search_tree.query(point)[1]

In [44]:
imp_data

,kcorr_g,w2,ra_obj,dec_obj,z,redshift
0,0.515064,17.720986,119.429449,52.995309,16.613500,0.169052
1,0.053598,18.739711,171.950334,34.842062,16.878486,0.042331
2,0.101875,16.903854,229.317800,10.181280,15.904148,0.059361
3,-0.029259,19.273824,2.593309,18.374851,19.933580,0.171086
4,0.139702,16.945460,223.510943,30.590311,16.847751,0.105039
...,...,...,...,...,...,...
1652055,0.608065,18.249756,41.771824,-26.326319,18.305663,0.039340
1652056,0.163659,NaN,2.118993,17.437530,15.273295,0.067905
1652057,0.870919,17.362712,10.077710,-29.285906,16.548414,0.264760
1652058,0.221352,16.726668,243.607075,52.124665,15.633123,0.089299


In [45]:
#getting spherical coords and taking glaxies with labels 
data_np = imp_data
#ra_rec= imp_data['ra_obj']
#dec_rec=imp_data['dec_obj']
#z_rec= imp_data['z']

#spher_coords=pd.read_csv('drive/MyDrive/rcsed_z_less_03_iGrID.csv')['iGrID']
#spher_coords=spher_coords.dropna()

#data_np=pd.DataFrame(np.load('drive/MyDrive/ra_dec_z.npy'), columns=['ra','dec','z'])


#getting spherical coords and taking glaxies with labels 
ra_rec= (data_np.ra_obj-data_np.ra_obj.min()) / (data_np.ra_obj.max()-data_np.ra_obj.min()) * 2 * np.pi
dec_rec=((data_np.dec_obj-data_np.dec_obj.min()) / (data_np.dec_obj.max()-data_np.dec_obj.min()) * np.pi)
z_rec= data_np.z
spher_coords=pd.DataFrame({'phi':ra_rec,'omega':dec_rec,'z':z_rec})
spher_coords['labels']=pd.read_csv('drive/MyDrive/iGrID_rcsed_8_clean.csv')['iGrID']
spher_coords=spher_coords[spher_coords.labels.notna()]

In [38]:
imp_data

,w2,ra_obj,dec_obj,z,redshift
0,17.720986,119.429449,52.995309,16.613500,0.169052
1,18.739711,171.950334,34.842062,16.878486,0.042331
2,16.903854,229.317800,10.181280,15.904148,0.059361
3,19.273824,2.593309,18.374851,19.933580,0.171086
4,16.945460,223.510943,30.590311,16.847751,0.105039
...,...,...,...,...,...
1652055,18.249756,41.771824,-26.326319,18.305663,0.039340
1652056,NaN,2.118993,17.437530,15.273295,0.067905
1652057,17.362712,10.077710,-29.285906,16.548414,0.264760
1652058,16.726668,243.607075,52.124665,15.633123,0.089299


In [46]:
#transform spherical coords to decart
x=spher_coords.z*np.sin(spher_coords.omega)*np.cos(spher_coords.phi)
y=spher_coords.z*np.sin(spher_coords.omega)*np.sin(spher_coords.phi)
z=spher_coords.z*np.cos(spher_coords.omega)
galaxies_with_labels=pd.DataFrame({'x':x,'y':y,'z':z, 'w2':imp_data['w2'], 'redshift':imp_data['redshift'], 'kcorr_g':imp_data['kcorr_g'], 'labels':spher_coords['labels']})

In [47]:
sample_galaxies=galaxies_with_labels.copy()

In [35]:
sample_galaxies = sample_galaxies[sample_galaxies['kcorr_g'].notna()]
sample_galaxies = sample_galaxies[sample_galaxies['w2'].notna()]

In [50]:
sample_galaxies = sample_galaxies.dropna()

In [51]:
sample_galaxies

,x,y,z,w2,redshift,kcorr_g,labels
59,12.991335,13.588780,-0.329969,19.649860,0.128075,-0.002799,50126.0
62,9.224510,-5.861885,13.896884,19.528216,0.077080,0.010315,129111.0
72,-9.436226,14.418607,-1.134260,17.830058,0.240597,0.089759,212738.0
92,-13.750424,-12.965265,0.218196,19.146099,0.235120,0.840002,51045.0
99,-14.649813,-9.679444,1.265266,17.996513,0.136330,0.127370,203801.0
...,...,...,...,...,...,...,...
1652040,16.058033,9.971290,1.886475,19.236226,0.138600,0.521290,85758.0
1652041,-8.185746,-5.029041,-12.713973,17.304942,0.082785,0.150567,55875.0
1652042,14.453764,-4.427546,9.150696,18.354782,0.081420,0.094957,62572.0
1652048,-13.890109,13.464372,-0.203613,20.230851,0.018770,1.220283,145300.0


In [57]:
max_fms=0
log=[]
for i in range(2, 20,5):
  for j in range(1,10,2):
    for k in range(10, 300,50):
      clusterer = hdbscan.hdbscan_.HDBSCAN(min_cluster_size=i, #2,
                                         min_samples=j, #3,
                                         cluster_selection_epsilon=0.0,
                                         metric='euclidean',
                                         p=None,
                                         algorithm='boruvka_kdtree',   #'boruvka_kdtree', 
                                         leaf_size=k, #30,
                                         approx_min_span_tree=True,
                                         gen_min_span_tree=True,
                                         core_dist_n_jobs=4,
                                         cluster_selection_method='eom',
                                         allow_single_cluster=False,
                                         prediction_data=False,
                                         match_reference_implementation=False).fit(sample_galaxies.drop(columns=['labels']))
      labels = clusterer.labels_
 
      for l in range(len(labels)):
        if labels[l]==-1:
          labels[l]=l+5000000
 
      true = sample_galaxies.labels
      pred = labels
 
      fms = round(fowlkes_mallows_score(true, pred),5)
      ars = round(adjusted_rand_score(true, pred),5)
      if max_fms<fms:
        max_fms=fms
      log.append([i,j,k,fms,ars])
      print('min_cluster_size =', i, ', min_samples =', j, ', leaf_size =', k, ', fms =', fms, ', ars =', ars)

min_cluster_size = 2 , min_samples = 1 , leaf_size = 10 , fms = 0.0008 , ars = 0.00071
min_cluster_size = 2 , min_samples = 1 , leaf_size = 60 , fms = 0.0008 , ars = 0.00071
min_cluster_size = 2 , min_samples = 1 , leaf_size = 110 , fms = 0.0008 , ars = 0.00071
min_cluster_size = 2 , min_samples = 1 , leaf_size = 160 , fms = 0.0008 , ars = 0.00071
min_cluster_size = 2 , min_samples = 1 , leaf_size = 210 , fms = 0.0008 , ars = 0.00071
min_cluster_size = 2 , min_samples = 1 , leaf_size = 260 , fms = 0.0008 , ars = 0.00071
min_cluster_size = 2 , min_samples = 3 , leaf_size = 10 , fms = 0.00108 , ars = 0.00099
min_cluster_size = 2 , min_samples = 3 , leaf_size = 60 , fms = 0.00109 , ars = 0.00101
min_cluster_size = 2 , min_samples = 3 , leaf_size = 110 , fms = 0.00112 , ars = 0.00103
min_cluster_size = 2 , min_samples = 3 , leaf_size = 160 , fms = 0.00112 , ars = 0.00103
min_cluster_size = 2 , min_samples = 3 , leaf_size = 210 , fms = 0.00112 , ars = 0.00103
min_cluster_size = 2 , min_samp